In [40]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_txt= f.read()

print("Total number of character:",len(raw_txt))
print(raw_txt[:99])

Total number of character: 20479
I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no 


In [41]:
import re
text = "Hello, world. This, is a test."
result= re.split(r'(\s)',text)
print(result)

['Hello,', ' ', 'world.', ' ', 'This,', ' ', 'is', ' ', 'a', ' ', 'test.']


In [42]:
text = "Hello, world. This, is a test."
result= re.split(r'([,.]|\s)',text)
print(result)

['Hello', ',', '', ' ', 'world', '.', '', ' ', 'This', ',', '', ' ', 'is', ' ', 'a', ' ', 'test', '.', '']


In [43]:
result=[item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', 'a', 'test', '.']


In [44]:
text = "Hello, world. This, is-- a test?"
result= re.split(r'([,.?":;_!\'()]|--|\s)',text)
result=[item for item in result if item.strip()]
print(result)

['Hello', ',', 'world', '.', 'This', ',', 'is', '--', 'a', 'test', '?']


In [45]:
preprocessed= re.split(r'([,.?":;_!\'()]|--|\s)',raw_txt)
preprocessed=[item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [46]:
print(len(preprocessed))

4690


In [47]:
all_words= sorted(set(preprocessed))
vocab_size= len(all_words)
print(vocab_size)

1130


In [48]:
vocab={token:integer for integer,token in enumerate(all_words)}
for i,item in enumerate(vocab.items()):
  print(item)
  if i>=50:
    break

('!', 0)
('"', 1)
("'", 2)
('(', 3)
(')', 4)
(',', 5)
('--', 6)
('.', 7)
(':', 8)
(';', 9)
('?', 10)
('A', 11)
('Ah', 12)
('Among', 13)
('And', 14)
('Are', 15)
('Arrt', 16)
('As', 17)
('At', 18)
('Be', 19)
('Begin', 20)
('Burlington', 21)
('But', 22)
('By', 23)
('Carlo', 24)
('Chicago', 25)
('Claude', 26)
('Come', 27)
('Croft', 28)
('Destroyed', 29)
('Devonshire', 30)
('Don', 31)
('Dubarry', 32)
('Emperors', 33)
('Florence', 34)
('For', 35)
('Gallery', 36)
('Gideon', 37)
('Gisburn', 38)
('Gisburns', 39)
('Grafton', 40)
('Greek', 41)
('Grindle', 42)
('Grindles', 43)
('HAD', 44)
('Had', 45)
('Hang', 46)
('Has', 47)
('He', 48)
('Her', 49)
('Hermia', 50)


In [49]:
class SimpleTokenizerV1:
  def __init__(self,vocab):
    self.str_to_int = vocab
    self.int_to_str = {i:s for s,i in vocab.items()}

  def encode(self,text):
    preprocessed= re.split(r'([,.?":;_!\'()]|--|\s)',text)
    preprocessed=[item for item in preprocessed if item.strip()]
    ids= [self.str_to_int[s] for s in preprocessed]
    return ids

  def decode(self,ids):
    text=" ".join([self.int_to_str[i] for i in ids])
    text = re.sub(r'\s+([.,?!()\'])',r'\1',text)
    return text


In [50]:
tokenizer= SimpleTokenizerV1(vocab)
text= """It's the last he painted, you know,"
Mrs, Gisburn said with pride."""
ids= tokenizer.encode(text)

In [51]:
print(ids)

[56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 5, 38, 851, 1108, 793, 7]


In [52]:
tokenizer.decode(ids)

'It\' s the last he painted, you know, " Mrs, Gisburn said with pride.'

**Adding** Special Context Tokens

In [53]:
all_tokens= sorted(list(set(preprocessed)))
all_tokens.extend(["<|endoftext|>","<|unk|>"])
vocab= {token:integer for integer,token in enumerate(all_tokens)}

In [54]:
len(vocab.items())

1132

In [55]:
for i,item in enumerate(list(vocab.items())[-5:]):
  print(item)

('younger', 1127)
('your', 1128)
('yourself', 1129)
('<|endoftext|>', 1130)
('<|unk|>', 1131)


In [56]:
class SimpleTokenizerV2:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = { i:s for s,i in vocab.items()}

    def encode(self, text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)', text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed = [
            item if item in self.str_to_int
            else "<|unk|>" for item in preprocessed
        ]

        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        # Replace spaces before the specified punctuations
        text = re.sub(r'\s+([,.:;?!"()\'])', r'\1', text)
        return text

In [57]:
tokenizer= SimpleTokenizerV2(vocab)
text1="Hello, do you like tea?"
text2="In the sunlit terraces of the palace."
text=" <|endoftext|> ".join((text1,text2))

In [58]:
print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [59]:
tokenizer.encode(text)

[1131, 5, 355, 1126, 628, 975, 10, 1130, 55, 988, 956, 984, 722, 988, 1131, 7]

In [60]:
tokenizer.decode(tokenizer.encode(text))

'<|unk|>, do you like tea? <|endoftext|> In the sunlit terraces of the <|unk|>.'

In [61]:
#BPE
#HANDLES OOV AND HAS LESS VOCAB SIZE
#SUBWORD TOKENIZATION

In [62]:
!pip3 install tiktoken

In [63]:
import tiktoken
tokenizer= tiktoken.get_encoding("gpt2")
text = (
    "Hello, do you like tea? <|endoftext|> In the sunlit terraces"
     "of someunknownPlace."
)
integer= tokenizer.encode(text,allowed_special={"<|endoftext|>"})
print(integer)
string = tokenizer.decode(integer)
print(string)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]
Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.


In [64]:
#data sampling with sliding window

In [65]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_txt= f.read()

encoded_txt= tokenizer.encode(raw_txt)
print(len(encoded_txt))

5145


In [66]:
context_size=10
x=encoded_txt[:context_size]
y=encoded_txt[1:context_size+1]
print(x,y)

[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138] [367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138, 257]


In [67]:
for i in range(1,context_size+1):
  context= encoded_txt[:i]
  target= encoded_txt[i]
  print(context,target)
  print(tokenizer.decode(context),tokenizer.decode([target]))

[40] 367
I  H
[40, 367] 2885
I H AD
[40, 367, 2885] 1464
I HAD  always
[40, 367, 2885, 1464] 1807
I HAD always  thought
[40, 367, 2885, 1464, 1807] 3619
I HAD always thought  Jack
[40, 367, 2885, 1464, 1807, 3619] 402
I HAD always thought Jack  G
[40, 367, 2885, 1464, 1807, 3619, 402] 271
I HAD always thought Jack G is
[40, 367, 2885, 1464, 1807, 3619, 402, 271] 10899
I HAD always thought Jack Gis burn
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899] 2138
I HAD always thought Jack Gisburn  rather
[40, 367, 2885, 1464, 1807, 3619, 402, 271, 10899, 2138] 257
I HAD always thought Jack Gisburn rather  a


In [81]:
# implement data loader
from torch.utils.data import Dataset,DataLoader
import torch
class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids=[]
    self.target_ids=[]

    token_ids= tokenizer.encode(txt,allowed_special={"<|endoftext|>"})
    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk= token_ids[i:i+max_length]
      target_chunk= token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
      return len(self.input_ids)

  def __getitem__(self,idx):
    return self.input_ids[idx],self.target_ids[idx]

In [82]:
def create_dataloader_v1(txt,batch_Size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):
  tokenizer= tiktoken.get_encoding("gpt2")
  dataset= GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader= DataLoader(dataset,batch_size=batch_Size,shuffle=shuffle,drop_last=drop_last,num_workers=num_workers)
  return dataloader

In [84]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_txt= f.read()

In [86]:
import torch
dataloader= create_dataloader_v1(raw_txt,batch_Size=4,max_length=20,stride=5,shuffle=True,drop_last=True,num_workers=0)
data_iter= iter(dataloader)
first_batch= next(data_iter)
print(first_batch)

[tensor([[  379,   790,   966,   262,  3512,   286, 14081,  2415,   284,   307,
         13055,   366, 11576,   306,     1,   780,   673,   373, 10032,   286],
        [ 1297,  9074,    13,   520,  5493,   523,   618,   673,  2540,   284,
           336,   321,   647,  1223,   546,   607,  8098,    13,   314,  3505],
        [   12,  5532, 14000,    13,   764,   764,   764,   198,   198,     1,
            40,  2900,   736,   284,   616,   670,    11,   290,  1816,   319],
        [ 2666,   572,  1701,   198,   198,  1544,  6204,   510,   290,  8104,
           465,  1021,   319,   616,  8163,   351,   257,  6487,    13,   366]]), tensor([[  790,   966,   262,  3512,   286, 14081,  2415,   284,   307, 13055,
           366, 11576,   306,     1,   780,   673,   373, 10032,   286,   852],
        [ 9074,    13,   520,  5493,   523,   618,   673,  2540,   284,   336,
           321,   647,  1223,   546,   607,  8098,    13,   314,  3505,  1972],
        [ 5532, 14000,    13,   764,   764,